In [162]:
import numpy as np
from rdkit import Chem

from interaction_components.plinteraction import get_interactions

In [163]:
def get_residue(mol_prot):
    all_residues = []
    for at in mol_prot.GetAtoms():
        resinfo = at.GetPDBResidueInfo()

        chainid = resinfo.GetChainId().strip()
        residue_type = resinfo.GetResidueName().strip()
        residue_number = str(resinfo.GetResidueNumber()).strip()
        if len(chainid) == 0:
            chainid = "A"
        #             raise RuntimeWarning("no chain id in pdb")
        label = "_".join([chainid, residue_type, residue_number])
        all_residues.append(label)
    all_residues = list(set(all_residues))
    all_residues.sort()
    return all_residues

In [164]:
def create_hb_dict(residues):
    interaction_dict = {}
    for name in residues:
        interaction_dict.update({name + "_side": 0})
        interaction_dict.update({name + "_main": 0})
    return interaction_dict

In [165]:
def calc_hbond_strength(hb):
    dist = hb.distance_ad
    return -(1 / (1 + np.power(dist / 2.6, 6))) / 0.58

In [203]:
def calc_hb(hbonds, hb_dict):
    for hb in hbonds:
        reschain = hb.reschain.strip()
        restype = hb.restype
        sidechain = hb.sidechain
        resnr = str(hb.resnr)
        if len(reschain) == 0:
            #             raise RuntimeWarning("no chain id in hbonds")
            reschain = "A"
        label = "_".join([reschain, restype, resnr])
        energy = calc_hbond_strength(hb)
        if sidechain:
            key = label + "_side"
        else:
            key = label + "_main"
        hb_dict[key] += energy
    return

In [204]:
def calc_hbonds_descriptor(interactions, protein_residues):
    hb_dict = create_hb_dict(protein_residues)
    calc_hb(interactions.hbonds_ldon, hb_dict)
    calc_hb(interactions.hbonds_pdon, hb_dict)
    return hb_dict

In [205]:
def calc_saltbridge_strength(saltbridge):
    dist = saltbridge.distance
    if dist <= 4.0:
        return 1
    else:
        return 5.0 - dist

In [206]:
def calc_saltbridge(saltbridges, sb_dict):
    for sb in saltbridges:
        reschain = sb.reschain.strip()
        restype = sb.restype
        resnr = str(sb.resnr)
        if len(reschain) == 0:
            #             raise RuntimeWarning("no chain id in saltbridge")
            reschain = "A"
        label = "_".join([reschain, restype, resnr])
        energy = calc_saltbridge_strength(sb)
        sb_dict[label] += energy
    return

In [207]:
def create_sb_dict(residues):
    interaction_dict = {}
    for name in residues:
        interaction_dict.update({name: 0})
    return interaction_dict

In [208]:
def calc_saltbridge_descriptor(interactions, protein_residues):
    sb_dict = create_sb_dict(protein_residues)
    calc_saltbridge(interactions.saltbridge_lneg, sb_dict)
    calc_saltbridge(interactions.saltbridge_pneg, sb_dict)
    return sb_dict

In [209]:
radius = {
    "N": 1.8,
    "O": 1.7,
    "S": 2.0,
    "P": 2.1,
    "F": 1.5,
    "Cl": 1.8,
    "Br": 2.0,
    "I": 2.2,
    "C": 1.9,
    "H": 0.0,
    "Zn": 0.5,
    "B": 1.8,
    "Si": 1.8,
    "As": 1.8,
    "Se": 1.8,
}


def get_hyd_strength(dist, patom, latom):
    p_radii = radius.get(patom.GetSymbol(), 0.5)
    l_radii = radius.get(latom.GetSymbol(), 0.5)

    d1 = p_radii + l_radii
    if dist <= d1 + 0.5:
        return -1.0
    elif d1 + 0.5 < dist < d1 + 2.0:
        return -0.66666 * (d1 + 2.0 - dist)
    else:
        return 0


def calc_hydrophobic(hydro):
    dist = hydro.distance
    patom = hydro.bsatom
    latom = hydro.ligatom

    hydrophobic_energy = get_hyd_strength(dist, patom, latom)
    return hydrophobic_energy

In [210]:
def create_hc_dict(residues):
    interaction_dict = {}
    for name in residues:
        interaction_dict.update({name + "_side": 0})
        interaction_dict.update({name + "_main": 0})
    return interaction_dict

In [211]:
def calc_hydrophybic_descriptor(interactions, protein_residues):
    hc_dict = create_hc_dict(protein_residues)
    for hc in interactions.all_hydrophobic_contacts:
        reschain = hc.reschain.strip()
        restype = hc.restype
        sidechain = hc.sidechain
        resnr = str(hc.resnr)
        if len(reschain) == 0:
            reschain = "A"
        #             raise RuntimeWarning("no chain id in hydrophybic")
        label = "_".join([reschain, restype, resnr])
        energy = calc_hydrophobic(hc)
        if sidechain:
            key = label + "_side"
        else:
            key = label + "_main"
        hc_dict[key] += energy
    return hc_dict

In [212]:
def calc_ifp(mol_prot, mol_lig):
    protein_residues = get_residue(mol_prot)
    result = get_interactions(mol_prot, mol_lig)
    interactions = result.interactions

    hbonds_fp = calc_hbonds_descriptor(interactions, protein_residues)
    saltbridge_fp = calc_saltbridge_descriptor(interactions, protein_residues)
    hc_fp = calc_hydrophybic_descriptor(interactions, protein_residues)
    return protein_residues, hbonds_fp, saltbridge_fp, hc_fp

In [233]:
protein_file = "data/5vsd/pocket.pdb"
ligand_file = "data/5vsd/Lig_fixed.sdf"

In [234]:
mol_prot = Chem.MolFromPDBFile(protein_file, removeHs=False)
mol_ligs = Chem.SDMolSupplier(ligand_file, removeHs=False)

In [235]:
for mol_lig in mol_ligs:
    protein_residues, hbonds_fp, saltbridge_fp, hc_fp = calc_ifp(mol_prot, mol_lig)